In [1]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
yolo_path = "C:/Users/navid.hejazi/OneDrive - Tensator/Desktop/YOLO/"

In [3]:
classes = []
with open("C:/Users/navid.hejazi/OneDrive - Tensator/Desktop/YOLO/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [4]:
# Define net 
yolo_net = cv.dnn.readNet(os.path.join(yolo_path, "yolov3.weights"), os.path.join(yolo_path, "yolov3.cfg"))
# Define layaers
layer_names = yolo_net.getLayerNames()
# Define output layers, 3 for yolo v3
output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]


In [5]:
image = cv.imread("Images/04_tehran.png")
height, width, channels = image.shape
image.shape

(1284, 1998, 3)

In [6]:
# (416, 416) input net size -  (True) swap BGR2RGB
img_blob = cv.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

In [7]:
# Detrmine input
yolo_net.setInput(img_blob)
outputs = yolo_net.forward(output_layers)
outputs[0].shape

(507, 85)

In [8]:
class_ids = []
confidences = []
boxes = []
for out in outputs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence> 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [10]:

colors = np.random.uniform(0, 255, size=(len(classes), 3))
colors[0],colors[1],colors[2],colors[5],colors[9] = (0, 136, 255), (226, 43, 138), (255, 191, 0), (153, 255, 255), (164, 203, 255)
indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        cv.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv.putText(image, label, (x, y + 30), cv.FONT_HERSHEY_PLAIN, 2, color, 2)

cv.imwrite("Images/04_tehran_det_cv.png", image)


True

In [99]:
# Object detection in Video

In [14]:
cap = cv.VideoCapture(0)

while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    img = cv.resize(frame, None, fx=0.8, fy=0.8)
    height, width, channels = img.shape
    blob = cv.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    outs = yolo_net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv.putText(img, label, (x, y + 30), font, 2, color, 2)
    cv.imshow('Frame',img)
    if cv.waitKey(25) & 0xFF == ord('q'):
      break
cap.release()
cv.destroyAllWindows()

[4]
[3]
[2]
[3]
[3 5]
[3]
[3]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[4]
[3]
[3]
[4]
[1]
[4]
[1]
[3]
[3]
[3]
[3]
[3]
[2]
[4]
[2 0]
[4 6]
[6 8]
[5 6]
[5 6]
[4 7]
[3 6]
[3 7]
[3 6]
[3 8 9]
[3 8 9]
[ 4  9 10]
[3 8]
[4]
[3]
[3 7 6]
[3 7]
[2 6]
[3]
[3 7]
[3]
[4 6]
[1]
[3]
[2 6 7]
[1 6 7]
[ 4 10]
[ 2 10]
[ 0 12]
[ 6 11]
[ 6 11]
[ 4 10 12]
[2 4]
[2 4]
[2 4]
[2 4]
[2 4]
[2 4]
[2 4 8]
[2 4]
[2 4]
[2 4 8]
[2 4 8]
[2 4 8]
[2 4 8]
[2 4 8]
[2 4 8]
[2 4 8]
[2 4 8]
[2 4 8]
[2 6 9]
[2 6 9]
[3 9]
[6 8 9]
[ 2 10 11]
[2]
[5]
[5]
[4]
[4 5]
[3 5]
[4 6]
[4 6 7]
[3 7]
[4 7 6]
[ 3  7 10]
[2 6]
[2 5 6]
[3 7 8]
[2 8]
[3 8 9]
[3 7 8]
[3 7 8]
[3 8 9]
[2 7 8]
[2 7 9]
[2 7 6]
[2 7 8]
[2 7 8]
[2 7 8]
[1 6]
[2 6]
[4]
[2 9 7 6]
[ 2  7 10]
[2 6]
[2 5 6]
[3 5 8]
[0 2 5]
[1 0 6]
[1 2 6]
[0 2 6]
[7 2 0 8 3]
[2 7 8]
[1 6 3]
[1 6]
[1 7 3]
[3]
[3]
[0]
[0 1]
[0 1]
[0 1]
[0 1]
[4 1]
[0]
[0 1]
[3]
[4 1]
[0 1]
[1 2 0]
[4 1]
[0]
[0 1]
[0 1]
[0]
[0 1]
[0 1]
[5 2 0]
[0 1]